In [1]:
%%capture
!pip install geopy meteostat

In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
from datetime import datetime, timedelta
from meteostat import Point, Daily, Stations, Hourly, Monthly
import matplotlib.pyplot as plt

### Obtenir les coordonnées des départements du Sénégal avec la bibliothèque geopy

In [3]:
data = {"Adresse Complète":[], 
        "Département":[], 
        "Latitude":[], 
        "Longitude":[],
        "Altitude":[]
        }

departements = [
    "Dakar", "Guédiawaye", "Pikine", "Rufisque", "Keur Massar",
    "Bambey", "Diourbel", "Mbacké",
    "Fatick", "Foundiougne", "Gossas",
    "Kaolack", "Guinguinéo", "Nioro du Rip",
    "Kolda", "Vélingara", "Médina Yoro Foulah",
    "Kébémer", "Linguère", "Louga",
    "Kanel", "Matam", "Ranérou",
    "Dagana", "Podor", "Saint-Louis",
    "Bakel", "Koumpentoum", "Tambacounda", "Goudiry",
    "Mbour", "Thiès", "Tivaouane",
    "Bignona", "Oussouye", "Ziguinchor",
    "Birkelane", "Kaffrine", "Malem-Hodar", "Koungheul",
    "Kédougou","Salemata", "Saraya",
    "Bounkiling", "Goudomp", "Sédhiou"
]

def get_coordonnees(list_dpt):
    for dpt in list_dpt:

        
        loc = Nominatim(user_agent="GetLoc")

        getLoc = loc.geocode(dpt+" departement, Sénégal")
        
        adress = getLoc.address.split(",")
        data.setdefault("Adresse Complète", []).append(getLoc.address)
        data.setdefault("Département", []).append(dpt)
        data.setdefault("Longitude", []).append(getLoc.longitude)
        data.setdefault("Latitude", []).append(getLoc.latitude)
        data.setdefault("Altitude", []).append(getLoc.altitude)

    return data

data = get_coordonnees(departements)
df = pd.DataFrame.from_dict(data, orient="columns")

### Obtenir la météo de ces départements durant les 12 derniers mois avec la bibliothèque meteostat

In [4]:
start = datetime.now() - timedelta(days=100)
end = datetime.now() + timedelta(days=1)

val1 = list(df.iloc[0,:])
city = Point(val1[2], val1[3], val1[4])


data1 = Monthly(city, start, end)
data1 = data1.fetch()
data1.insert(0, "Address", val1[0])
data1.insert(1, "Department", val1[1])
data1.insert(2, "Latitude", val1[2])
data1.insert(3, "Longitude", val1[3])
data1.insert(4, "Altitude", val1[4])


for i in range(1, df.shape[0]):
  val = list(df.iloc[i,:])
  city = Point(val[2], val[3], val[4])

  data = Monthly(city, start, end)
  data = data.fetch()
  #data.drop("Région", axis=1)
  data.insert(0, "Address", val[0])
  data.insert(1, "Department", val[1])
  data.insert(2, "Latitude", val[2])
  data.insert(3, "Longitude", val[3])
  data.insert(4, "Altitude", val[4])

  data1 = data1.append(data)
  #print(data.head(5))

In [5]:
final_df = data1.sort_index()
final_df.head()

,Address,Department,Latitude,Longitude,Altitude,tavg,tmin,tmax,prcp,wspd,pres,tsun
2022-08-01,"Kaolack, Département de Kaolack, Région de Kao...",Kaolack,14.138815,-16.076391,0.0,28.0,24.6,32.1,195.6,9.6,1011.3,NaN
2022-08-01,"Guinguinéo, Département de Guinguinéo, Région ...",Guinguinéo,14.272179,-15.945573,0.0,28.0,24.6,32.1,195.6,9.6,1011.3,NaN
2022-08-01,"Kolda, Département de Kolda, Région de Kolda, ...",Kolda,12.893608,-14.939658,0.0,27.3,23.8,32.2,411.7,7.2,1010.4,NaN
2022-08-01,"Tambacounda, Département de Tambacounda, Regió...",Tambacounda,13.769258,-13.668290,0.0,27.7,23.9,32.8,215.9,8.9,1011.2,NaN
2022-08-01,"Bignona, Département de Bignona, Région de Zig...",Bignona,12.801133,-16.228970,0.0,27.1,23.7,32.6,568.1,7.4,1011.8,NaN


In [6]:
final_df.to_csv("/content/drive/MyDrive/SALTIS-IAS Data Collect/Weather.csv")

In [7]:
data1 = final_df.sort_index()
data1 = data1.rename_axis('Date').reset_index()
data1.to_dict('list')

{'Date': [Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00')],
 'Address': ['Kaolack, Département de Kaolack, Région de Kaolack, 24000, Sénégal',
  'Guinguinéo, Département de Guinguinéo, Région de Kaolack, Sénégal',
  'Kolda, Département de Kolda, Région de Kolda, Sénégal',
  'Tambacounda, Département de Tambacounda, Región de Tambacounda, 26000, Sénégal',
  'Bignona, Département de Bignona, Région de Ziguinchor, Sénégal',
  "Oussouye, Département d'Oussouye, Région de Ziguinchor, Sénégal",
  'Ziguinchor, Département de Ziguinchor, Région de Ziguinchor, Sénégal',
  'Birkelane, Département de Birkelane, Région de Kaffrine, Sénégal',
  'Kédougou, Département de Kédougou, Région de Kédougou, 26005, Sénégal'],
 'Department'

### Explication des colonnes du datasets


| variables | definitions |
| --- | --- | 
| Date | La date aaaa-mm-jj | 
| Address | L'emplacement exact |
| Latitude | La latitude du lieu |
| Longitude | La longitude du lieu |
| Altitude | L'altitude du lieu |
| Department | Le département dans lequel se trouve le lieux |
| tavg | La température moyenne de l'air en °C |
| tmin | La température minimale de l'air en °C |
| tmax | La température maximale de l'air en °C |
| prcp | Le total des précipitations quotidiennes en mm |
| snow | L'épaisseur maximale de neige en mm |
| wdir | La direction moyenne du vent en °C |
| wspd | La vitesse moyenne du vent en °C |
| wpgt | La rafale de vent maximale en km/h |
| pres | La pression atmosphérique moyenne au niveau de la mer en hPa |
| tsun | L'ensoleillement quotidien total en minutes (m) |